### Want to see: 
- Still looking at change in neural activity projected onto preference axes
- Is the magnitude of this projection different for different belief partitions?
- Prediction for chose X, correct: 
  - low: larger
  - High X: lower, maybe negative
  - High not X: lower
- Prediction for chose X, incorrect: 
  - low, lower
  - High X: large negative 
  - High not X: lower




In [1]:
import os
import numpy as np
import pandas as pd
import utils.visualization_utils as visualization_utils

import utils.io_utils as io_utils

import utils.glm_utils as glm_utils
import utils.classifier_utils as classifier_utils
from matplotlib import pyplot as plt
import matplotlib
import utils.spike_utils as spike_utils
from constants.behavioral_constants import *
from constants.decoding_constants import *
from scripts.pseudo_decoding.belief_partitions.belief_partition_configs import *
import scripts.pseudo_decoding.belief_partitions.belief_partitions_io as belief_partitions_io
import itertools

import argparse
import copy
from tqdm import tqdm
import seaborn as sns

In [4]:
conditions_maps = {
    "chose X / correct": {"Response": "Correct", "Choice": "Chose"},
    "chose X / incorrect": {"Response": "Incorrect", "Choice": "Chose"},

    # "correct": {"Response": "Correct"},
    # "incorrect": {"Response": "Incorrect"},
}

def read_all_cond_data(args):
    all_data = []
    for i, event in enumerate(["StimOnset", "FeedbackOnsetLong"]):
        all_conds = []
        for cond_name in conditions_maps:
            args.trial_event = event
            args.conditions = conditions_maps[cond_name]
            res = belief_partitions_io.read_update_projections(args)
            res["cond"] = res.apply(lambda x: "shuffle" if "shuffle" in x["mode"] else cond_name, axis=1)
            all_conds.append(res)
        all_conds = pd.concat(all_conds)
        all_conds["trial_event"] = event
        all_conds["Time"] = all_conds["TimeIdx"] / 10 + 0.1
        all_data.append(all_conds)
    return pd.concat(all_data)

In [11]:
args = argparse.Namespace(
    **BeliefPartitionConfigs()._asdict()
)
args.subject = "both"
args.beh_filters = {"Response": "Correct", "Choice": "Chose"}
args.base_output_path = "/data/patrick_res/update_projections"
args.mode = "pref"
args.sig_unit_level = f"{args.mode}_99th_window_filter_drift"


In [12]:
res = read_all_cond_data(args)

In [13]:
res

,TimeIdx,PseudoTrialNumber,proj,mode,feat,cond,trial_event,Time
0,-10,0,3.273248,pref,CIRCLE,chose X / correct,StimOnset,-0.9
1,-10,1,-4.744916,pref,CIRCLE,chose X / correct,StimOnset,-0.9
2,-10,2,16.421106,pref,CIRCLE,chose X / correct,StimOnset,-0.9
3,-10,3,-3.702267,pref,CIRCLE,chose X / correct,StimOnset,-0.9
4,-10,4,-2.065434,pref,CIRCLE,chose X / correct,StimOnset,-0.9
...,...,...,...,...,...,...,...,...
32995,14,995,0.382805,pref_shuffle,SWIRL,shuffle,FeedbackOnsetLong,1.5
32996,14,996,1.800568,pref_shuffle,SWIRL,shuffle,FeedbackOnsetLong,1.5
32997,14,997,-36.211657,pref_shuffle,SWIRL,shuffle,FeedbackOnsetLong,1.5
32998,14,998,3.324514,pref_shuffle,SWIRL,shuffle,FeedbackOnsetLong,1.5
